In [ ]:
import pandas as pd

df = pd.read_csv("original.csv", encoding="ISO-8859-1", low_memory=False)

df = df[["iyear", "imonth", "iday", "attacktype1_txt"]].dropna(subset=["iyear", "imonth", "attacktype1_txt"])

df["iday"] = df["iday"].replace(0, 1)

df["date"] = pd.to_datetime(
    dict(year=df["iyear"], month=df["imonth"], day=df["iday"]),
    errors="coerce"
)
df = df.dropna(subset=["date"])

agg = (
    df.groupby([pd.Grouper(key="date", freq="YS"), "attacktype1_txt"])
    .size()
    .reset_index(name="attack_count")
)

pivot = agg.pivot(
    index="date",
    columns="attacktype1_txt",
    values="attack_count"
).fillna(0)

N = 5
top_types = pivot.sum().sort_values(ascending=False).head(N).index
pivot_top = pivot[top_types]

SMOOTH_WINDOW = 5 

pivot_smooth = pivot_top.rolling(
    window=SMOOTH_WINDOW,
    center=True,
    min_periods=1
).mean()

cumsum = pivot_smooth.cumsum()

export_df = cumsum.reset_index()

export_df['date'] = export_df['date'].dt.strftime('%Y')

output_filename = "d3_cumulative_data_yearly.csv"
export_df.to_csv(output_filename, index=False)

print(f"Success! Yearly data exported to {output_filename}")
print(f"Smoothing Window: {SMOOTH_WINDOW} years")
print("First 5 rows:")
print(export_df.head())